In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import pickle

In [2]:
data = pd.read_csv("data/df.csv", index_col=0)
noun_dist = np.load(f"data/lda_noun_distribution.npy")
adj_dist = np.load(f"data/lda_adj_distribution.npy")

with open(f"data/lda_topics.pickle", "rb") as f:
    topics = pickle.load(f)

In [3]:
data.shape, adj_dist.shape, noun_dist.shape

((5817, 7358), (1000, 7358), (5817, 1000))

#### Katsetus leida tõenäosused, et kohv on kange või tugev

In [4]:
data.head()

,eelmine,järgmine,viimane,käesolev,kogu,kohalik,suur,pikk,juriidiline,tulev,...,realiseeritav,valgustatav,vajuv,väikesekasvuline,väikeseformaadiline,õlirikas,õigusetu,oletuslik,raugematu,raudteeäärne
aasta,53250,38107,24410,11753,1115,40,114,1562,0,12634,...,0,0,0,0,0,0,0,0,0,0
aeg,75,65,27915,1227,24997,1373,194,14913,0,18,...,0,0,0,0,0,0,0,0,0,0
määrus,16,110,17,27514,30,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
omavalitsus,30,67,44,0,31,24261,84,0,12,0,...,0,0,0,0,0,0,0,0,0,0
osa,201,436,918,118,51,10,22165,25,0,17,...,0,0,0,0,0,0,0,0,0,0


In [4]:
list(data.index).index('kohv')

1505

In [5]:
kohv_topicud = noun_dist[1505]

In [6]:
list(data.columns).index('kange'), list(data.columns).index('tugev')

(127, 73)

#### Top 50 klastrit

Leian sõnale 'kohv' vastavad 50 kõige tõenäolisemat klastrit

In [7]:
ind = np.argpartition(kohv_topicud, -50)[-50:]

In [8]:
ind = ind[np.argsort(kohv_topicud[ind])][::-1]

Loon numpy array, kuhu salvestan genereeritud $\theta$ väärtusi

In [9]:
kohv_topics_theta = np.zeros((1000, 50, adj_dist.shape[1]))

In [10]:
kohv_topics_theta.shape

(1000, 50, 7358)

In [11]:
adj_dist[0, :].shape

(7358,)

In [12]:
kohv_topics_theta[:, 0].shape

(1000, 7358)

Iga klastri kohta genereerin $\theta$ vektori vastavast $\alpha$ vektorist, kus $\alpha$ on ühele klastrile vastav vektor, mis sisaldab omadussõnadele vastavaid pseudocounte.

In [13]:
for i in range(kohv_topics_theta.shape[1]):
    kohv_topics_theta[:, i] = scipy.stats.dirichlet.rvs(adj_dist[ind[i], :], size=1000)

Leian sõnale 'kohv' vastava $\beta$ vektori, mis on vektor, mis sisaldab iga klastri kohta tõenäosust, et sõna kuulub sellesse klastrisse. Kuna eelnevalt on kasutatud vaid top 50 klastrit, võtan ka siin vaid neile top 50 klastrile vastavad tõenäosused.

In [14]:
kohv_topicud_top = noun_dist[1505][ind]

Korrutan sõnale 'kohv' vastava tõenäosusvektori eelnevalt leitud $\theta$ vektoritega. $\theta$ vektorid on salvestatud andmestruktuuri kujul (N, num_topics, num_adjs), kus N on simulatsioonide arv (hetkel 1000), num_topics on vastav klastrite arv (siin võetud top 50 klastrit) ja num_adjs on ühele $\alpha$ vektorile vastava vektori pikkus ehk omadussõnade arv (siin 7358).

Korrutatud on igale klastrile vastav $\theta$ vektor $\beta$ vektoriga, seega tulemus on kujul (N, num_adjs) = (1000, 7358), mis tähendab, et iga simulatsiooni kohta on eraldi korrutis.

In [15]:
np.matmul(kohv_topicud_top, kohv_topics_theta).shape

(1000, 7358)

Leian keskmise kõikide N simulatsiooni peale.

In [16]:
np.mean(np.matmul(kohv_topicud_top, kohv_topics_theta), axis=0).shape

(7358,)

In [17]:
avg_per_adj = np.mean(np.matmul(kohv_topicud_top, kohv_topics_theta), axis=0)

Katsetused, et näha, mis erinevatele omadussõnadele vastavad väärtused on ning mis on üleüldiselt top omadussõnad sõna 'kohv' jaoks.

In [18]:
print(data.columns[127], avg_per_adj[127])
print(data.columns[73], avg_per_adj[73])

kange 0.0886423167761788
tugev 0.00047375078501561094


In [19]:
np.amax(avg_per_adj), np.argmax(avg_per_adj), data.columns[11]

(0.14438539629134167, 11, 'hea')

In [20]:
top_adj_ind = np.argpartition(avg_per_adj, -10)[-10:]
top_adj_ind = top_adj_ind[np.argsort(avg_per_adj[top_adj_ind])][::-1]

In [21]:
data.columns[top_adj_ind]

Index(['hea', 'must', 'terviklik', 'kuum', 'kange', 'lahustuv', 'valmis',
       'tavaline', 'väike', 'suur'],
      dtype='object')